<a href="https://colab.research.google.com/github/lephuocdat2000/DeepLearning-and-Application/blob/main/Final_Version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install simplejpeg

     |████████████████████████████████| 276kB 31.2MB/s 


In [ ]:
# Tạo môi trường để sử dung Yolo
!pip install opencv-python==4.4.0.46 -i https://pypi.douban.com/simple

Looking in indexes: https://pypi.douban.com/simple
     |████████████████████████████████| 49.5MB 60kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30


In [ ]:
#Thư viện web trên colab
!pip install flask-ngrok

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from operator import itemgetter
import itertools
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
from base64 import b64decode, b64encode
import cv2
import PIL
import io
import html
import time
%matplotlib inline

In [ ]:
def load_network_1(config_file, data_file, weights, batch_size=1):
    """
    load model description and weights from config files
    args:
        config_file (str): path to .cfg model file
        data_file (str): path to .data model file
        weights (str): path to weights
    returns:
        network: trained model
        class_names
        class_colors
    """
    network = load_net_custom(
        config_file.encode("ascii"),
        weights.encode("ascii"), 0, batch_size)
    metadata = load_meta(data_file.encode("ascii"))
    class_names = [metadata.names[i].decode("ascii") for i in range(1)]
    colors = class_colors(class_names)
    return network, class_names, colors

def darknet_helper(img, width, height):
  """
    predict pedesetrian and return boxes 
  """
  darknet_image = make_image(width, height, 3)
  img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img_resized = cv2.resize(img_rgb, (width, height),
                              interpolation=cv2.INTER_LINEAR)

  # get image ratios to convert bounding boxes to proper size
  img_height, img_width, _ = img.shape
  width_ratio = img_width/width
  height_ratio = img_height/height

  # run model on darknet style image to get detections
  copy_image_from_bytes(darknet_image, img_resized.tobytes())
  detections = detect_image(network, class_names, darknet_image)
  free_image(darknet_image)
  boxes = np.array([bbox2points(bbox) for _,_,bbox in detections])
  boxes = boxes * [width_ratio, height_ratio, width_ratio, height_ratio]
  return boxes

def MatrixCreation(A,B,C,D):
    """
    Create transformed matrix from 4 points: A,B,C,D 
    return matrix,width and height of bird-eye-image
    """
    width = int(max(np.sqrt((B[1]-A[1])**2+(B[0]-A[0])**2),np.sqrt((D[1]-C[1])**2+(D[0]-C[0])**2)))
    height = int(max(np.sqrt((C[1]-A[1])**2+(C[0]-A[0])**2),np.sqrt((D[1]-B[1])**2+(D[0]-B[0])**2)))
    inputs = np.float32([A,B,C,D])
    outputs = np.float32([[0,0],
                      [width-1,0],
                      [0,height-1],
                      [width-1,height-1]])
    M = cv2.getPerspectiveTransform(inputs,outputs)
    return M,width,height

def Points_Transformation(boxes):
    """
    Select bottom-center point in boxes and apply perspectiveTransform to points
    return transformed_points
    """
    x_ = boxes[:,0]+(boxes[:,2]-boxes[:,0]) / 2
    y_ = boxes[:,3]
    x_ = np.expand_dims(x_,axis = 1)
    y_ = np.expand_dims(y_,axis = 1) 
    # centroids = np.uint32(np.concatenate((x_,y_),axis=1))
    centroids = np.concatenate((x_,y_),axis=1)
    list_point_to_detect = centroids.reshape(-1,1,2)
    transformed_points = cv2.perspectiveTransform(list_point_to_detect, M)
    transformed_points = transformed_points.reshape(transformed_points.shape[0],transformed_points.shape[2])
    return transformed_points

def DrawnRectangel(boxes,image,point,color):
    xmin,ymin,xmax,ymax = boxes[point][:].astype(int)
    cv2.rectangle(image,(int(xmin),int(ymin)),(int(xmax),int(ymax)),color,2)  

def get_birds_eye_view_image(transformed_points,green_box,red_box,eye_view_height,eye_view_width):
    """
    Create bird_eye_view 
    """
    blank_image = cv2.imread('/content/drive/MyDrive/Advanced-CV/black_background.png')
    blank_image = cv2.resize(blank_image,(eye_view_width,eye_view_height))
    cv2.putText(blank_image, str(len(red_box)), (120,100), cv2.FONT_HERSHEY_SIMPLEX , 2, (0,0,255), 4, cv2.LINE_AA) 
    cv2.putText(blank_image, str(len(green_box)), (520,100), cv2.FONT_HERSHEY_SIMPLEX , 2, (0,255,0), 4, cv2.LINE_AA)
    for point in green_box:
        cv2.circle(blank_image,tuple(transformed_points[point].astype(int)),20,(0,255,0),-1)
    for point in red_box:
        cv2.circle(blank_image,tuple(transformed_points[point].astype(int)),20,(0,0,255),-1)
    blank_image = cv2.resize(blank_image,(eye_view_width//2,eye_view_height))
    return blank_image
     
def video_processing(video_path,real_width,real_height,pixel_width,pixel_height):
    cap = cv2.VideoCapture(video_path)
    if (cap.isOpened()==False): 
        print('Error opening video stream or file')
    else: 
      blank_image = cv2.imread('/content/drive/MyDrive/Advanced-CV/black_background.png')
      blank_image = cv2.resize(blank_image,(960,540))
      fourcc = cv2.VideoWriter_fourcc(*'DIVX')
      out = cv2.VideoWriter('output.avi',fourcc, 20, (1440,540))
      while (True):
          ret, frame = cap.read()
          if ret==True:
            alter_blank_image = blank_image.copy()
            boxes = darknet_helper(frame,width,height)  
            transformed_points = Points_Transformation(boxes)
            image_rect = frame.copy()
            if len(boxes) > 1:  
                list_indexes = list(itertools.combinations(range(len(transformed_points)), 2))
                x_y_ = []
                for i,pair in enumerate(itertools.combinations(transformed_points, r=2)):
                     if np.sqrt( ((pair[0][0] - pair[1][0])*real_width/pixel_width)**2 + ((pair[0][1] - pair[1][1])*real_height/pixel_height)**2  ) < int(distance_minimum):
                         index_pt1 = list_indexes[i][0]
                         index_pt2 = list_indexes[i][1]
                         if index_pt1 not in x_y_:
                            #draw red circle top view
                            cv2.circle(alter_blank_image,tuple(transformed_points[index_pt1].astype(int)),20,(0,0,255),-1)
                            #change red box original frame
                            DrawnRectangel(boxes,image_rect,index_pt1,(0,0,255))
                            x_y_.append(index_pt1)
                         if index_pt2 not in x_y_: 
                            #draw red circle top view    
                            cv2.circle(alter_blank_image,tuple(transformed_points[index_pt2].astype(int)),20,(0,0,255),-1)
                            #change red box original frame    
                            DrawnRectangel(boxes,image_rect,index_pt2,(0,0,255))
                            x_y_.append(index_pt2) 
                #select points to draw green box and green circle 
                diff = np.setdiff1d(list(range(0,len(boxes))),np.unique(x_y_))
                for i in diff:
                   DrawnRectangel(boxes,image_rect,i,(0,255,0))
                   cv2.circle(alter_blank_image,tuple(transformed_points[i].astype(int)),20,(0,255,0),-1)
            elif len(boxes)==1: 
                 DrawnRectangel(boxes,image_rect,0,(0,255,0))
                 cv2.circle(alter_blank_image,tuple(transformed_points[0].astype(int)),20,(0,255,0),-1)
            #Write nums of red
            height_blank,width_blank = alter_blank_image.shape[0],alter_blank_image.shape[1]
            cv2.putText(alter_blank_image, str(len(x_y_)), (int(width_blank*0.225),int(height_blank*0.1)), cv2.FONT_HERSHEY_SIMPLEX , 2, (0,0,255), 4, cv2.LINE_AA) 
            #Write nums of green
            cv2.putText(alter_blank_image, str(len(diff)), (int(width_blank*0.7),int(height_blank*0.1)), cv2.FONT_HERSHEY_SIMPLEX , 2, (0,255,0), 4, cv2.LINE_AA)
            alter_blank_image = cv2.resize(alter_blank_image,(alter_blank_image.shape[1]//2,alter_blank_image.shape[0]))
            combined_image = np.concatenate((alter_blank_image,image_rect),axis=1)
            out.write(combined_image)
            if cv2.waitKey(1)==ord('q'): break
          else: break 
    out.release()
    return
def show_webcam(mirror=False):
    vid = cv2.VideoCapture(0)
    while True:
       ret,frame = vid.read()
       plt.imshow('frame',frame)
       if cv2.waitKey(1) & 0xFF == ord('q'):
          break
    vid.release()
    cv2.destroyAllWindows() 


In [ ]:
%cd /content/drive/MyDrive/darknet
#608X608
# load in our YOLOv4 architecture network
from darknet import *
network, class_names, class_colors = load_network_1("cfg/yolov4.cfg", "cfg/coco.data", "yolov4.weights")
width = network_width(network)
height = network_height(network)


/content/drive/MyDrive/darknet


In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, jsonify, render_template,Response
import sys
from imutils.video import FileVideoStream
import simplejpeg

app = Flask(__name__,template_folder="/content/templates")
run_with_ngrok(app)
@app.route('/')
def home():
    return render_template('index.html')
@app.route('/detectObject' , methods=['POST'])
def mask_image():
	file = request.files['image'] ## byte file
	file.save("/content/data.avi")
	video=cv2.VideoCapture("/content/data.avi")
	ret,frame=video.read()
	if not ret:
		return jsonify({'error':True})
	return jsonify({'success':True})
####################### MONITOR SOCIAL DISTANCE ############################
image_height=540
image_width=960
#4 points ROI
A = [571,151]
B = [768,185]
C = [283,400]
D = [690,521]
M,pixel_width,pixel_height = MatrixCreation(A,B,C,D)
real_width,real_height = 4.0,7.6
distance_minimum = 2.0

def gen():
    frame_number = 0
    video = cv2.VideoCapture('/content/data.avi')
    min_distance=200
    #blank_image is background of bird-eye-view
    blank_image = cv2.imread('/content/black_background.png')
    blank_image = cv2.resize(blank_image,(pixel_width,pixel_height))
    while True:
        ret,frame = video.read()
        if not ret:
          break
        #Phát hiện người
        alter_blank_image = blank_image.copy()
        #Predict boxes
        boxes = darknet_helper(frame,width,height) 
        # Transform bottom-center points 
        transformed_points = Points_Transformation(boxes)
        image_rect = frame.copy()
        # Calculate distance and drawn rectangle, circle
        if len(boxes) > 1:  
            list_indexes = list(itertools.combinations(range(len(transformed_points)), 2))
            x_y_ = []
            for i,pair in enumerate(itertools.combinations(transformed_points, r=2)):
               if np.sqrt( ((pair[0][0] - pair[1][0])*real_width/pixel_width)**2 + ((pair[0][1] - pair[1][1])*real_height/pixel_height)**2  ) < int(distance_minimum):
                  index_pt1 = list_indexes[i][0]
                  index_pt2 = list_indexes[i][1]
                  if index_pt1 not in x_y_:
                     #draw red circle top view
                     cv2.circle(alter_blank_image,tuple(transformed_points[index_pt1].astype(int)),20,(0,0,255),-1)
                     #change red box original frame
                     DrawnRectangel(boxes,image_rect,index_pt1,(0,0,255))
                     x_y_.append(index_pt1)
                  if index_pt2 not in x_y_: 
                     #draw red circle top view    
                     cv2.circle(alter_blank_image,tuple(transformed_points[index_pt2].astype(int)),20,(0,0,255),-1)
                     #change red box original frame    
                     DrawnRectangel(boxes,image_rect,index_pt2,(0,0,255))
                     x_y_.append(index_pt2) 
               #select points to draw green box and green circle 
               diff = np.setdiff1d(list(range(0,len(boxes))),np.unique(x_y_))
               for i in diff:
                   DrawnRectangel(boxes,image_rect,i,(0,255,0))
                   cv2.circle(alter_blank_image,tuple(transformed_points[i].astype(int)),20,(0,255,0),-1)
        elif len(boxes)==1: 
            DrawnRectangel(boxes,image_rect,0,(0,255,0))
            cv2.circle(alter_blank_image,tuple(transformed_points[0].astype(int)),20,(0,255,0),-1)
        #Write nums of red
        height_blank,width_blank = alter_blank_image.shape[0],alter_blank_image.shape[1]
        cv2.putText(alter_blank_image, str(len(x_y_)), (int(width_blank*0.225),int(height_blank*0.1)), cv2.FONT_HERSHEY_SIMPLEX , 2, (0,0,255), 4, cv2.LINE_AA) 
        #Write nums of green
        cv2.putText(alter_blank_image, str(len(diff)), (int(width_blank*0.7),int(height_blank*0.1)), cv2.FONT_HERSHEY_SIMPLEX , 2, (0,255,0), 4, cv2.LINE_AA)
        alter_blank_image = cv2.resize(alter_blank_image,(image_width//2,image_height))
        combined_image = np.concatenate((alter_blank_image,image_rect),axis=1)
        frame=simplejpeg.encode_jpeg(image=combined_image,quality=70,colorspace='BGR',colorsubsampling='444', fastdct=True)
        yield (b'--frame\r\n'b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')
        del boxes,alter_blank_image,diff,x_y_,combined_image
        key = cv2.waitKey(5)
        if key == 30:
          break
@app.route('/video_feed')
def video_feed():
	return Response(gen(), mimetype='multipart/x-mixed-replace; boundary=frame')
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://d04e576db121.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [23/Jun/2021 10:02:20] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Jun/2021 10:02:21] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [23/Jun/2021 10:02:22] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Jun/2021 10:02:35] "POST /detectObject HTTP/1.1" 200 -
127.0.0.1 - - [23/Jun/2021 10:02:35] "GET /video_feed HTTP/1.1" 200 -
